In [ ]:
!git clone https://github.com/panwarp/PyL3dMD.git

In [34]:
import sys
import os

# モジュールパス追加
sys.path.append(os.path.abspath('./PyL3dMD/pyl3dmd'))
import math
import numpy as np
np.math = math #descriptors2setでnp.math.が登場する

import getaway
import descriptors1set
import descriptors2set
import descriptors3set
import descriptors4set
import descriptors5set
import descriptors6set
from getatomicproperties import getatomicproperties
from getinfofromlammpsdatafile import caladjacencymatrix,buildangleslist,builddihedralslist
import getadjacencyanddistancematrices
import numpy as np
from scipy import sparse
from ase.io import Trajectory,read,write
from ase.neighborlist import NeighborList, natural_cutoffs
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore', category=RuntimeWarning)


In [35]:
###############################################################################
# (A) ASEのtrajから分子情報を抜き出す: eachMolsIdx,eachMolsBonds,eachMolsAngles,eachMolsDihedrals,eachMolsMass
###############################################################################
def build_molecule_dicts(atoms):

    # (A) 全原子の質量を取得 (shape=(N,))
    all_masses = atoms.get_masses()

    # (B) natural_cutoffsを計算して NeighborList を構築 (現状は高圧バルクでも分子数を判断できている)
    cutoffs = natural_cutoffs(atoms)
    nl = NeighborList(cutoffs, self_interaction=False, bothways=True)
    nl.update(atoms)

    # (C) スパースな接続行列を取得 -> connected_components
    connectivity_matrix = nl.get_connectivity_matrix(sparse=True).tocsr()
    n_components, labels = sparse.csgraph.connected_components(connectivity_matrix)

    # (D) 各分子ID -> グローバル原子インデックス
    eachMolsIdx = {i: [] for i in range(n_components)}
    for atom_index, component_id in enumerate(labels):
        eachMolsIdx[component_id].append(atom_index)

    # (E) 分子ごとのbonds, angles, dihedrals, mass を構築
    eachMolsBonds = {}
    eachMolsAngles = {}
    eachMolsDihedrals = {}
    eachMolsMass = {}

    for mol_id, atom_indices in eachMolsIdx.items():
        # グローバル -> ローカルのマッピングを作成
        local_map = {g_idx: i_local for i_local, g_idx in enumerate(atom_indices)}

        # (E1) bondsを抽出（1-basedの局所インデックスに変換）
        bond_list = []
        for g_i in atom_indices:
            for g_j in connectivity_matrix[g_i].indices:
                if g_j in atom_indices and g_j > g_i:
                    li = local_map[g_i] + 1
                    lj = local_map[g_j] + 1
                    bond_list.append([li, lj])
        bonds_local = np.array(bond_list, dtype=int)
        eachMolsBonds[mol_id] = bonds_local

        # (E2) angles, dihedrals を計算するために
        #      まずは 0-based の adjacency を作成
        adjMatrix, adjList = caladjacencymatrix(bonds_local)

        angles_list = buildangleslist(adjList)       # 0-based shape (nAngles, 3)
        dihedrals_list = builddihedralslist(adjList) # 0-based shape (nDihedrals, 4)

        # 1-based に変換
        eachMolsAngles[mol_id] = angles_list + 1
        eachMolsDihedrals[mol_id] = dihedrals_list + 1

        # (E3) 質量配列
        eachMolsMass[mol_id] = all_masses[atom_indices]


    # (F) まとめて返す
    return eachMolsIdx,eachMolsBonds,eachMolsAngles,eachMolsDihedrals,eachMolsMass


def calgeometricdistancematrix(xyz):
        """
        ユークリッド距離の計算 (numAtoms x numAtoms)
        """
        onesMat = np.ones([len(xyz), len(xyz)])
        Gx = onesMat * xyz[:, 0] - np.transpose(onesMat * xyz[:, 0])
        Gy = onesMat * xyz[:, 1] - np.transpose(onesMat * xyz[:, 1])
        Gz = onesMat * xyz[:, 2] - np.transpose(onesMat * xyz[:, 2])

        # Geometric Distance Matrix
        G = np.sqrt(Gx ** 2 + Gy ** 2 + Gz ** 2)
        return G

def caldensity(massBox, volBox):
        """
        密度の算出 [g/cc]
        """
        NA = 6.0221408e+23  #[1/mol]
        volBoxcc = volBox * 1.0E-24  # セル体積 [cc]

        rho = (massBox / volBoxcc) * (1.0 / NA)
        return rho

###############################################################################
# (B) 指定した.trajのNフレームに対して特徴量を計算
###############################################################################

def Calculate_descriptors(trajfile, last_n=10, whichdescriptors='all'):

    traj = Trajectory(trajfile, mode='r')
    n_frames = len(traj)
    start_frame = max(0, n_frames - last_n)

    results = []

    for frame_idx in range(start_frame, n_frames):
        atoms = traj[frame_idx]

        # (1) 分子情報を取得 -> eeachMolsIdx,eachMolsBonds,eachMolsAngles,eachMolsDihedrals,eachMolsMass
        eachMolsIdx,eachMolsBonds,eachMolsAngles,eachMolsDihedrals,eachMolsMass = build_molecule_dicts(atoms)
        eachMolsAdjMat, eachMolsDisMat = getadjacencyanddistancematrices.getadjANDdismatrices(eachMolsMass, eachMolsBonds)
        charges = atoms.get_charges()
        if charges is None:
            print("Warning: No partial charges found. Set charges to 0.")
            charges = np.zeros_like(eachMolsMass)
        all_masses = atoms.get_masses()

        #getatomicproperties.pyから計算
        (eachMolsZ, 
         eachMolsL,
         eachMolsZv,
         eachMolsRv,
         eachMolsRc,
         eachMolsm,
         eachMolsV,
         eachMolsEn,
         eachMolsalapha,
         eachMolsIP,
         eachMolsEA) = getatomicproperties(all_masses, eachMolsIdx) #半分くらいしか使わないやんけ

        positions = atoms.get_positions()  # (N,3)

        molDescriptors = {}
        numMols = len(eachMolsIdx)

        mass_per_atom = atoms.get_masses()  # shape (N,)
        total_mass_amu = np.sum(mass_per_atom)

        rho = []
        for i in range(n_frames):
            atoms_i = traj[i]
            volume_A3 = atoms_i.get_volume()  # ASE が計算するセル体積 (Å^3)
            rho_i = caldensity(total_mass_amu, volume_A3)  # g/cc
            rho.append(rho_i)
        
        print(f"frame:{frame_idx}  系内の分子数: {len(eachMolsIdx)}")

        for mol_id in range(numMols):
            # 指定した分子について情報を取り出す 
            g_indices = eachMolsIdx[mol_id]
            mass_local = eachMolsMass[mol_id]
            angles_local = eachMolsAngles[mol_id]
            dihedrals_local = eachMolsDihedrals[mol_id]
            density_local = rho[mol_id]
            disMat_local = eachMolsDisMat[mol_id]
            adjMat_local = eachMolsAdjMat[mol_id]

            xyz_local = positions[g_indices, :]
            c_local    = charges[g_indices]
            m_local  = mass_local

            # getatomicproperties の結果から取り出したプロパティ
            V_local  = eachMolsV[mol_id]
            En_local = eachMolsEn[mol_id]
            P_local  = eachMolsalapha[mol_id]
            IP_local = eachMolsIP[mol_id]
            EA_local = eachMolsEA[mol_id]

            bonds_local = eachMolsBonds[mol_id]  # 1-based
            G_local = calgeometricdistancematrix(xyz_local)
            
            # 分子のid,タイムフレーム、密度
            others = {
                'molecule': mol_id + 1,
                'Timeframe': frame_idx,
                'rho': density_local
            }

            # ---- whichdescriptors で分岐して計算 ----

            if whichdescriptors == 'all':

                # set6
                RDF, ATS, GATS, MATS, MoRSE = descriptors6set.getragmmdescriptors(
                    G_local, c_local, m_local, V_local, En_local, P_local, IP_local, EA_local
                )
                RDF, ATS, GATS, MATS, MoRSE = descriptors6set.getragmmdescriptors(
                    G_local, c_local, m_local, V_local, En_local, P_local, IP_local, EA_local
                )
                # set5
                WHIM = descriptors5set.getwhimdescriptors(
                    xyz_local, c_local, m_local, V_local, En_local, P_local, IP_local, EA_local
                )
                # set4
                CPSA = descriptors4set.getcpsadescriptors(
                    xyz_local, c_local, eachMolsRc[mol_id]
                )
                # set3
                GETAWAY = descriptors3set.getgetawayhatsindexes(
                    xyz_local, mass_local, bonds_local,
                    c_local, m_local, V_local, En_local, P_local, IP_local, EA_local
                )
                # set2
                GMdes, DDMdes = descriptors2set.calgeometricdescriptors(
                    xyz_local, mass_local, c_local, bonds_local, angles_local, 
                    dihedrals_local, density_local, disMat_local
                    )
                # set1
                TCdes = descriptors1set.caltopologyconnectivitydescriptors(
                    xyz_local, mass_local, bonds_local, angles_local, 
                    dihedrals_local, adjMat_local, disMat_local
                    )

                # まとめる
                res = {**others,**TCdes,  **GMdes, **DDMdes,**GETAWAY,  **CPSA,
                        **WHIM,**RDF, **ATS,  **MATS, **MoRSE} #**GATSは返さない　元コードでもバグあり
                
            elif whichdescriptors == 'set1':
                # set1 のみ
                TCdes = descriptors1set.caltopologyconnectivitydescriptors(
                    xyz_local, mass_local, bonds_local, angles_local, 
                    dihedrals_local, adjMat_local, disMat_local
                    )
                res = {**others, **TCdes}
            
            elif whichdescriptors == 'set2':
                # set2
                GMdes, DDMdes = descriptors2set.calgeometricdescriptors(
                    xyz_local, mass_local, c_local, bonds_local, angles_local, 
                    dihedrals_local, density_local, disMat_local
                    )
                res = {**others, **GMdes, **DDMdes}

            elif whichdescriptors == 'set3':
                # set3 のみ
                GETAWAY = descriptors3set.getgetawayhatsindexes(
                    xyz_local, mass_local, bonds_local,
                    c_local, m_local, V_local, En_local, P_local, IP_local, EA_local
                )
                res = {**others, **GETAWAY}

            elif whichdescriptors == 'set4':
                # set4 => CPSA
                CPSA = descriptors4set.getcpsadescriptors(
                    xyz_local, c_local, eachMolsRc[mol_id]
                )
                res = {**others, **CPSA}

            elif whichdescriptors == 'set5':
                # set5 => WHIM
                WHIM = descriptors5set.getwhimdescriptors(
                    xyz_local, c_local, m_local, V_local, En_local, P_local, IP_local, EA_local
                )
                res = {**others, **WHIM}

            elif whichdescriptors == 'set6':
                # set6 => getragmmdescriptors
                RDF, ATS, GATS, MATS, MoRSE = descriptors6set.getragmmdescriptors(
                    G_local, c_local, m_local, V_local, En_local, P_local, IP_local, EA_local
                )
                res = {**others, **RDF, **ATS, **MATS, **MoRSE} #**GATSは返さない　元コードでもバグあり

            else:
                print(f"Warning: '{whichdescriptors}' not implemented. Skipped.")
                res = {**others} 

            molDescriptors[mol_id] = res

        # フレームごとにまとめる
        frame_res = {
            'frame': frame_idx,
            'molDescriptors': molDescriptors
        }
        results.append(frame_res)

    return results

def Run_pyl3dmd_asetraj(trajfile, last_n=10, output_csv=None, whichdescriptors='all'):

    # トラジェから特徴量を計算
    results_all = Calculate_descriptors(trajfile, last_n,whichdescriptors)

    # 各フレームごとに処理
    rows = []
    for frame_result in results_all:
        frame_idx = frame_result['frame']
        for mol_id, desc_dict in frame_result['molDescriptors'].items():
            row = {}
            for k, v in desc_dict.items():
                row[k] = v
            rows.append(row)

    df = pd.DataFrame(rows)

    if output_csv is not None:
        df.to_csv(output_csv, index=False)
        print(f"Saved results to {output_csv}")



In [36]:
Run_pyl3dmd_asetraj("./data/hexadecane_NPT_313.15K_1013250000Pa.traj", 10,"output.csv","set1")

frame:90  系内の分子数: 10
frame:91  系内の分子数: 10
frame:92  系内の分子数: 10
frame:93  系内の分子数: 10
frame:94  系内の分子数: 10
frame:95  系内の分子数: 10
frame:96  系内の分子数: 10
frame:97  系内の分子数: 10
frame:98  系内の分子数: 10
frame:99  系内の分子数: 10
Saved results to output.csv


In [33]:
df = pd.read_csv("output.csv")
df

,molecule,Timeframe,rho,delta3D,sigma3D,sigmaAvg3D,W3D,WA3D,gfactor,Wlike3D,...,XuI13D,XuI23D,chiR3D,chi03D,chi13D,chi23D,chi33D,X03D,X13D,GMTI
0,1,90,1.027792,46.525445,1371.703630,85.731477,685.851815,5.715432,1.008606,3.172206e+06,...,17.822228,17.815346,1.434156,9.566532,5.110529,14.263129,1.432998,4.159363,1.016641,2288.794421
1,2,90,0.763369,46.128495,1361.443244,85.090203,680.721622,5.672680,1.001061,1.397268e+06,...,17.805964,17.806971,1.445178,9.590839,5.146394,15.097286,1.462761,4.159216,1.016452,2277.797008
2,3,90,0.909621,45.845410,1320.950065,82.559379,660.475032,5.503959,0.971287,2.626296e+05,...,17.632692,17.631502,1.472603,9.628301,5.181920,14.502665,1.479286,4.163006,1.018458,2222.454295
3,4,90,0.909549,46.212050,1164.645629,72.790352,582.322815,4.852690,0.856357,1.314483e+05,...,17.166417,17.168485,1.672536,9.579965,5.133909,14.635025,1.462389,4.149666,1.012181,1987.430650
4,5,90,0.890851,46.196995,1382.758419,86.422401,691.379209,5.761493,1.016734,1.000672e+06,...,17.840739,17.846242,1.415948,9.580539,5.136161,15.028193,1.462832,4.163712,1.018701,2316.131445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6,99,0.922389,45.896409,1421.136992,88.821062,710.568496,5.921404,1.044954,1.389502e+06,...,17.948231,17.948772,1.376725,9.623282,5.173907,13.741508,1.477122,4.166532,1.020203,2383.767224
96,7,99,0.924862,45.912491,1307.859010,81.741188,653.929505,5.449413,0.961661,9.747445e+05,...,17.624833,17.632033,1.499705,9.615625,5.169236,14.446399,1.481040,4.156389,1.015150,2188.846710
97,8,99,0.930592,45.845489,1350.851675,84.428230,675.425838,5.628549,0.993273,4.675664e+05,...,17.729851,17.727857,1.443052,9.639374,5.183356,14.934076,1.479668,4.165447,1.019615,2269.135372
98,9,99,0.948504,45.913028,1174.303753,73.393985,587.151876,4.892932,0.863459,1.852921e+05,...,17.219281,17.220822,1.668430,9.616529,5.170164,14.945764,1.476930,4.147050,1.010613,1988.789351
